In [ ]:
# Step 1: Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [7]:
# Step 2: Load and preprocess the dataset

import cv2

# Define paths to the dataset directories
yes_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes"
no_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no"

# Define image dimensions
img_width, img_height = 150, 150

# Function to load and preprocess images from a directory
def load_images_from_directory(directory):
    image_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".JPG"):
            img = cv2.imread(os.path.join(directory, filename))
            img = cv2.resize(img, (img_width, img_height))
            img = img / 255.0  # Normalize pixel values to [0,1]
            image_list.append(img)
    return image_list

# Load images from directories
yes_images = load_images_from_directory(yes_dir)
no_images = load_images_from_directory(no_dir)

# Convert lists to numpy arrays
yes_images = np.array(yes_images)
no_images = np.array(no_images)

# Print the number of images loaded from each directory
print("Number of 'YES' images:", len(yes_images))
print("Number of 'NO' images:", len(no_images))


Number of 'YES' images: 154
Number of 'NO' images: 97


In [8]:
# Step 3: Define the CNN architecture

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize the CNN
model = Sequential()

# Add convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Step 4: Train the model

# Concatenate the images and labels
X = np.concatenate((yes_images, no_images), axis=0)
y = np.concatenate((np.ones(len(yes_images)), np.zeros(len(no_images))), axis=0)

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 762ms/step - accuracy: 0.6459 - loss: 0.6753 - val_accuracy: 0.6275 - val_loss: 0.6797
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 779ms/step - accuracy: 0.7915 - loss: 0.4690 - val_accuracy: 0.6667 - val_loss: 0.8776
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 709ms/step - accuracy: 0.8085 - loss: 0.4969 - val_accuracy: 0.6863 - val_loss: 0.6205
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 703ms/step - accuracy: 0.8229 - loss: 0.4552 - val_accuracy: 0.6667 - val_loss: 0.8186
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 702ms/step - accuracy: 0.8477 - loss: 0.3765 - val_accuracy: 0.6863 - val_loss: 0.7571
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 695ms/step - accuracy: 0.8364 - loss: 0.3810 - val_accuracy: 0.7059 - val_loss: 0.7014
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 688ms/step - accuracy: 0.8642 - loss: 0.3862 - val_accuracy: 0.7255 - val_loss: 0.6254
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 773ms/step - accuracy: 0.8517 - loss: 0.3246 - val_accuracy: 0.7451 - val_loss

In [18]:
# Save the model
model.save('BrainTumorDetection_CNN.h5')

print("Model saved successfully.")


Model saved successfully.
